<a href="https://colab.research.google.com/github/Rizvi999/neural-lab/blob/main/Exp13_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import time
import unicodedata
import re

print("="*70)
print("Neural Machine Translation with Attention Mechanism (OPTIMIZED)")
print("English to French Translator")
print("="*70)

# Download and prepare data
print("\n📥 Downloading dataset...")
path_to_zip = tf.keras.utils.get_file(
    'fra-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip',
    extract=True)

import os
import glob

# Find the extracted txt file
dataset_dir = os.path.dirname(path_to_zip)
txt_files = glob.glob(os.path.join(dataset_dir, '**/*.txt'), recursive=True)
if txt_files:
    path_to_file = txt_files[0]
    print(f"✅ Found dataset at: {path_to_file}")
else:
    extracted_dir = path_to_zip.replace('.zip', '_extracted')
    txt_files = glob.glob(os.path.join(extracted_dir, '*.txt'))
    if txt_files:
        path_to_file = txt_files[0]
        print(f"✅ Found dataset at: {path_to_file}")
    else:
        raise FileNotFoundError("Could not find the fra-eng.txt file")

# Load and preprocess data
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()
    w = '<start> ' + w + ' <end>'
    return w

def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]] for l in lines[:num_examples]]
    return zip(*word_pairs)

# OPTIMIZATION 1: Reduce dataset size to 10K samples (3x faster)
print("📚 Loading training data...")
en, fr = create_dataset(path_to_file, 10000)
en = list(en)
fr = list(fr)
print(f"✅ Loaded {len(en)} sentence pairs")

# Create tokenizers
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

print("🔤 Creating tokenizers...")
input_tensor, inp_lang = tokenize(en)
target_tensor, targ_lang = tokenize(fr)

max_length_inp = input_tensor.shape[1]
max_length_targ = target_tensor.shape[1]

print(f"English vocab size: {len(inp_lang.word_index)}")
print(f"French vocab size: {len(targ_lang.word_index)}")
print(f"Max input length: {max_length_inp}")
print(f"Max target length: {max_length_targ}")

# OPTIMIZATION 2: Larger batch size for faster training
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 128  # Increased from 64
steps_per_epoch = len(input_tensor) // BATCH_SIZE

# OPTIMIZATION 3: Reduced model complexity
embedding_dim = 128  # Reduced from 256
units = 512  # Reduced from 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

# Split data
from sklearn.model_selection import train_test_split
input_train, input_val, target_train, target_val = train_test_split(
    input_tensor, target_tensor, test_size=0.1, random_state=42)

# OPTIMIZATION 4: Add prefetch and caching for faster data loading
train_dataset = tf.data.Dataset.from_tensor_slices((input_train, target_train))
train_dataset = (train_dataset
                 .cache()  # Cache dataset in memory
                 .shuffle(BUFFER_SIZE)
                 .batch(BATCH_SIZE, drop_remainder=True)
                 .prefetch(tf.data.AUTOTUNE))  # Prefetch next batches

print(f"\n✅ Dataset prepared!")
print(f"Training samples: {len(input_train)}")
print(f"Validation samples: {len(input_val)}")

# Encoder with attention
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# Bahdanau Attention
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Decoder with attention
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

# Initialize models
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

# OPTIMIZATION 5: Use mixed precision training for faster computation
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

# Training step
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

# Training loop
print("\n" + "="*70)
print("🚀 TRAINING MODEL WITH ATTENTION (OPTIMIZED)")
print("="*70)
print("⚡ Optimizations applied:")
print("   • Reduced dataset: 10K samples (vs 30K)")
print("   • Smaller model: 512 units (vs 1024)")
print("   • Larger batches: 128 (vs 64)")
print("   • Data pipeline: cache + prefetch")
print("   • Mixed precision training enabled")
print("\nEstimated training time: ~2-3 minutes\n")

# OPTIMIZATION 6: Reduce epochs to 5
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 20 == 0:  # Print more frequently
            print(f'Epoch {epoch+1}/{EPOCHS} Batch {batch}/{steps_per_epoch} Loss {batch_loss.numpy():.4f}')

    epoch_time = time.time() - start
    avg_loss = total_loss / steps_per_epoch
    print(f'✅ Epoch {epoch+1} completed: Loss {avg_loss:.4f} | Time: {epoch_time:.1f}s\n')

print("✅ Training complete!\n")

# Translation function
def translate_sentence(sentence):
    sentence = preprocess_sentence(sentence)
    inputs = [inp_lang.word_index.get(i, 0) for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()

        word = targ_lang.index_word.get(predicted_id, '')

        if word == '<end>':
            break

        if word != '<start>':
            result += word + ' '

        dec_input = tf.expand_dims([predicted_id], 0)

    return result.strip()

# Sample translations
print("="*70)
print("📝 SAMPLE TRANSLATIONS")
print("="*70)

print("\nValidation set examples:\n")
for i in range(5):
    idx = np.random.randint(0, len(input_val))
    inp_sentence = ' '.join([inp_lang.index_word.get(j, '') for j in input_val[idx] if j != 0])
    real_translation = ' '.join([targ_lang.index_word.get(j, '') for j in target_val[idx] if j != 0])
    predicted_translation = translate_sentence(inp_sentence.replace('<start>', '').replace('<end>', '').strip())

    print(f"{i+1}.")
    print(f"   🇬🇧 English:    {inp_sentence.replace('<start>', '').replace('<end>', '').strip()}")
    print(f"   🇫🇷 Reference:  {real_translation.replace('<start>', '').replace('<end>', '').strip()}")
    print(f"   🤖 Translation: {predicted_translation}\n")

# Test custom sentences
print("="*70)
print("Testing common phrases:")
print("="*70 + "\n")

test_sentences = [
    "hello",
    "good morning",
    "thank you",
    "how are you",
    "i am happy",
    "what is your name",
    "nice to meet you",
    "goodbye"
]

for sent in test_sentences:
    translation = translate_sentence(sent)
    print(f"🇬🇧 {sent:25} → 🇫🇷 {translation}")

# Calculate accuracy
print("\n" + "="*70)
print("📊 CALCULATING TRANSLATION ACCURACY")
print("="*70)

def calculate_bleu_score(num_samples=100):
    correct_words = 0
    total_words = 0
    exact_matches = 0

    for i in range(min(num_samples, len(input_val))):
        inp_sentence = ' '.join([inp_lang.index_word.get(j, '') for j in input_val[i] if j != 0])
        inp_sentence = inp_sentence.replace('<start>', '').replace('<end>', '').strip()

        real_translation = ' '.join([targ_lang.index_word.get(j, '') for j in target_val[i] if j != 0])
        real_translation = real_translation.replace('<start>', '').replace('<end>', '').strip()

        predicted_translation = translate_sentence(inp_sentence)

        ref_words = set(real_translation.split())
        pred_words = set(predicted_translation.split())

        correct_words += len(ref_words & pred_words)
        total_words += len(ref_words)

        if real_translation.lower() == predicted_translation.lower():
            exact_matches += 1

    word_accuracy = (correct_words / total_words * 100) if total_words > 0 else 0
    exact_match_accuracy = (exact_matches / num_samples * 100)

    return word_accuracy, exact_match_accuracy

word_acc, exact_acc = calculate_bleu_score(100)

print(f"\n📈 Translation Quality (100 validation samples):")
print(f"   • Word Overlap Accuracy: {word_acc:.2f}%")
print(f"   • Exact Match Accuracy: {exact_acc:.2f}%")
print(f"   • Model: Seq2Seq with Bahdanau Attention (Optimized)")
print(f"   • Training samples: 10,000")
print(f"   • Epochs: 5")

# Interactive translator
print("\n" + "="*70)
print("🎯 INTERACTIVE TRANSLATOR")
print("="*70)
print("Enter English text to translate to French (or 'quit' to exit):\n")

while True:
    user_input = input("🇬🇧 English: ").strip()
    if user_input.lower() in ['quit', 'exit', 'q', '']:
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
        break

    translation = translate_sentence(user_input)
    print(f"🇫🇷 French:  {translation}\n")

Neural Machine Translation with Attention Mechanism (OPTIMIZED)
English to French Translator

📥 Downloading dataset...
3423204/3423204 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Found dataset at: /root/.keras/datasets/fra-eng_extracted/fra.txt
📚 Loading training data...
✅ Loaded 10000 sentence pairs
🔤 Creating tokenizers...
English vocab size: 2099
French vocab size: 3850
Max input length: 8
Max target length: 15

✅ Dataset prepared!
Training samples: 9000
Validation samples: 1000

🚀 TRAINING MODEL WITH ATTENTION (OPTIMIZED)
⚡ Optimizations applied:
   • Reduced dataset: 10K samples (vs 30K)
   • Smaller model: 512 units (vs 1024)
   • Larger batches: 128 (vs 64)
   • Data pipeline: cache + prefetch
   • Mixed precision training enabled

Estimated training time: ~2-3 minutes

Epoch 1/5 Batch 0/78 Loss 3.0571
Epoch 1/5 Batch 20/78 Loss 1.9502
Epoch 1/5 Batch 40/78 Loss 1.7940
Epoch 1/5 Batch 60/78 Loss 1.6985
✅ Epoch 1 completed: Loss 1.7380 | Time: 437.9s

Epoch 2/5 Batch 0/78 Loss 1.5923
Epoc